In [ ]:
# --------------------------------------------------------------
# Cell 1 : Imports & Global Configuration
# --------------------------------------------------------------

# %matplotlib inline is a magic command specific to Jupyter notebooks.
# It tells Jupyter to display matplotlib plots inline (within the notebook),
# instead of in a separate external window.
%matplotlib inline

# ==============================================================
# Core Python libraries for data science, mathematics, etc.
# ==============================================================
import numpy as np               # Numerical computing library, fundamental for arrays and math operations
import pandas as pd              # Data manipulation and analysis library
import matplotlib.pyplot as plt  # Library for generating plots and figures
import os                        # Provides a way of using operating system dependent functionality (e.g., file paths)
import glob                      # Used to find file paths matching a specified pattern
import time                      # Provides time-related functions (e.g., for measuring durations)
from datetime import datetime    # Allows handling of dates and times in a convenient way
import requests                  # Allows sending HTTP requests (used here for Horizons API calls)
import math                      # Math module for constants and mathematical functions

# ==============================================================
# FITS and astronomy-specific tools
# ==============================================================
from astropy.io import fits                         # For reading/writing FITS files (astronomical data)
from astropy.visualization import (ZScaleInterval,ImageNormalize)   # Helps with image normalization/scaling in astronomy
from astropy.wcs import WCS                         # World Coordinate System (used for astronomical image coordinates)
from photutils.detection import DAOStarFinder        # Used to detect stars or point sources in an image
from photutils.aperture import (aperture_photometry,CircularAperture)    # Used for photometric measurements in apertures
from astroquery.vizier import Vizier                # Allows querying of catalogs from the Vizier service
from reproject import reproject_interp               # Image reprojection, e.g., aligning images to a common WCS
from astroalign import register                      # Automatically align two astronomical images
from lightkurve import search_lightcurvefile         # For working with Kepler/TESS light curve data
import aplpy                                       # For advanced plotting of FITS images (astronomy)

# ==============================================================
# Image processing libraries
# ==============================================================
from skimage import filters, measure, morphology  # Various image processing algorithms (e.g., thresholding, morphological ops)
from skimage.measure import label, regionprops    # Tools to label and measure regions in binary images
from skimage.morphology import binary_closing, disk
from matplotlib.patches import Ellipse           # Allows drawing ellipse shapes on matplotlib figures
import matplotlib.pyplot as plt                  # (Imported again, but typically we'd have it once; duplicates are harmless)
import cv2                                       # OpenCV for advanced image processing operations

# ==============================================================
# Plotly for interactive visualizations
# ==============================================================
import plotly.graph_objects as go  # For creating interactive plots with Plotly

# --------------------------------------------------------------
# Constants used in this project
# --------------------------------------------------------------
horizons_url = 'https://ssd.jpl.nasa.gov/api/horizons_file.api'
# horizons_url: The base URL endpoint for NASA JPL Horizons API requests,
# which retrieves ephemeris data for solar system bodies, spacecraft, etc.


In [ ]:
# --------------------------------------------------------------------------------
# Cell 2 : Timer Utilities
# --------------------------------------------------------------------------------
# This section defines a set of utility functions to help measure and report the
# execution time of different sections of a script. The utilities are especially useful
# for performance profiling and debugging, as they allow you to track how long each part
# of your code takes to execute.

# --------------------------------------------------------------------------------
# Initialize a global list to store timing records for each section of code.
# Each record in the list will be a dictionary containing:
#   - 'section': A string label for the code section.
#   - 'start'  : A timestamp (float) when the section started.
#   - 'end'    : A timestamp (float) when the section ended (initially None).
#   - 'duration': The computed duration of the section (in seconds, initially None).
time_records = []  # This list serves as the central repository for all timing data.

# --------------------------------------------------------------------------------
# Define a function to mark the beginning of a new section of code that you wish to time.
def start_section(section_name):
    """
    Record the start time of a code section.

    Parameters:
    -----------
    section_name : str
        A descriptive name or label for the code section whose start time is being recorded.

    Process:
    --------
    1. Capture the current time using time.time(), which returns the time in seconds since the epoch.
    2. Create a new dictionary (i.e., a timing record) with the following keys:
         - 'section': Stores the provided section name.
         - 'start'  : Stores the current timestamp captured in step 1.
         - 'end'    : Initialized to None, as the section has not yet completed.
         - 'duration': Initialized to None, since the elapsed time has not been calculated yet.
    3. Append this dictionary to the global list 'time_records' for later use.
    
    Example:
    --------
    To time a data loading process, you might call:
        start_section("Data Loading")
    """
    # Get the current time (in seconds since the epoch) and store it in the variable 'now'.
    now = time.time()  # 'time.time()' returns a floating-point number representing the current time.
    
    # Append a new dictionary to the global 'time_records' list.
    # This dictionary encapsulates all necessary timing details for the current section.
    time_records.append({
        'section': section_name,  # Assign the provided section name to the 'section' key.
        'start': now,             # Record the current timestamp under the 'start' key.
        'end': None,              # Initialize the 'end' key to None; it will be updated when the section ends.
        'duration': None          # Initialize the 'duration' key to None; will be computed later.
    })
    # End of the start_section function.

# --------------------------------------------------------------------------------
# Define a function to mark the end of the most recent section and compute its duration.
def end_section():
    """
    Record the end time for the most recent section and compute its duration.

    Process:
    --------
    1. Capture the current time as the ending timestamp using time.time().
    2. Check if there is at least one timing record in the 'time_records' list.
       If the list is empty (i.e., no section was started), the function returns immediately.
    3. Retrieve the most recent timing record from the list (i.e., the last dictionary).
    4. Update the 'end' key in this record with the current timestamp.
    5. Calculate the duration by subtracting the 'start' time from the current time, and update
       the 'duration' key with this value.
       
    Example:
    --------
    After completing a timed code section, you would call:
        end_section()
    This marks the end of the most recently started section.
    """
    # Capture the current time, which will be used as the end time for the section.
    now = time.time()  # 'now' is a float representing the current timestamp.
    
    # Check if there are any timing records in the 'time_records' list.
    # If the list is empty, then no section has been started, and we have nothing to end.
    if not time_records:
        return  # Exit the function early if 'time_records' is empty.
    
    # Retrieve the most recent timing record (i.e., the last entry in the list).
    record = time_records[-1]  # This is the timing record corresponding to the most recent section.
    
    # Update the 'end' key of the retrieved record with the current timestamp.
    record['end'] = now  # Mark the time at which the section ended.
    
    # Compute the duration for the section by subtracting the recorded 'start' time from the current time.
    # The resulting duration (in seconds) is stored in the 'duration' key.
    record['duration'] = now - record['start']
    # End of the end_section function.

# --------------------------------------------------------------------------------
# Define a function to print a detailed summary of the timing information for all recorded sections.
def print_time_summary():
    """
    Print a structured table summarizing timing details for each recorded section,
    as well as the total runtime for the entire process.

    Output Details:
    ---------------
    - Each row in the table corresponds to a recorded section and displays:
         * The section name.
         * The start time formatted as HH:MM:SS.ms (milliseconds precision).
         * The end time formatted in the same manner (or "N/A" if not available).
         * The duration in seconds, formatted to three decimal places (or "N/A" if not computed).
    - At the end of the table, the total runtime is printed. This is calculated as the difference
      between the earliest start time (from the first section) and the latest end time (from the last section).
    
    Process:
    --------
    1. Check if there are any timing records; if none exist, the function returns without printing.
    2. Print a header to denote the beginning of the timing summary.
    3. Calculate the width for the 'Section' column by determining the length of the longest section name.
    4. Print a header row with column titles for Section, Start, End, and Duration.
    5. Draw a separator line for visual clarity.
    6. Compute the total runtime as the difference between the end time of the last section
       and the start time of the first section.
    7. Iterate over each timing record, formatting the start and end times from timestamps to human-readable
       strings (using the HH:MM:SS.ms format). If a section hasn't ended, "N/A" is displayed.
    8. Print each section's details in a formatted row.
    9. Print a final separator and the total runtime at the bottom of the summary.
    
    Example:
    --------
    After timing multiple sections of your script, simply call:
        print_time_summary()
    to display the timing summary table.
    """
    # First, check if there are any timing records available to print.
    if not time_records:
        return  # If 'time_records' is empty, exit the function without printing anything.
    
    # Print an introductory header for the timing summary.
    print("\n=== PROCESSING TIME SUMMARY ===")

    # --------------------------------------------------------------------------------
    # Determine the appropriate width for the 'Section' column in the output table.
    # This is calculated based on the length of the longest section name recorded.
    max_len = max(len(r['section']) for r in time_records)  # Find the maximum length of the section names.
    # Ensure a minimum width of 20 characters for the column, adding a little extra padding (+2) if needed.
    name_col_width = max(20, max_len + 2)

    # --------------------------------------------------------------------------------
    # Print the header row for the timing summary table.
    # The formatting instructions:
    #   - The section name is left-aligned within a width specified by 'name_col_width'.
    #   - The start and end times are right-aligned in fields 15 characters wide.
    #   - The duration (in seconds) is right-aligned in a field 14 characters wide.
    print(f"{'Section':<{name_col_width}}  {'Start':>15}   {'End':>15}   {'Duration (s)':>14}")
    
    # Print a separator line that spans the combined width of the table for clarity.
    print("-" * (name_col_width + 50))

    # --------------------------------------------------------------------------------
    # Calculate the total runtime of the entire process.
    # This is computed as the difference between the end time of the last section and the start time of the first section.
    total_time = time_records[-1]['end'] - time_records[0]['start']

    # --------------------------------------------------------------------------------
    # Loop through each timing record to print the details for each section.
    for r in time_records:
        # Extract the section name from the current timing record.
        section = r['section']
        
        # Convert the recorded 'start' timestamp (a float) into a formatted string:
        #   - datetime.fromtimestamp() converts the timestamp to a datetime object.
        #   - strftime("%H:%M:%S.%f") formats the datetime object into a string with hours, minutes, seconds, and microseconds.
        #   - The slicing [:-3] trims the last three digits of microseconds to display milliseconds instead.
        start_dt = datetime.fromtimestamp(r['start']).strftime("%H:%M:%S.%f")[:-3]
        
        # Similarly, convert the 'end' timestamp to a formatted string if it exists.
        # If the 'end' key is None (i.e., the section did not end properly), display "N/A" instead.
        end_dt = datetime.fromtimestamp(r['end']).strftime("%H:%M:%S.%f")[:-3] if r['end'] else "N/A"
        
        # Format the 'duration' value to three decimal places.
        # If 'duration' is None (i.e., not computed), display "N/A" as the output.
        duration = f"{r['duration']:.3f}" if r['duration'] else "N/A"

        # Print the details for this section in a formatted table row.
        # The formatting ensures that all columns (Section, Start, End, Duration) are aligned properly.
        print(f"{section:<{name_col_width}}  {start_dt:>15}   {end_dt:>15}   {duration:>14}")

    # --------------------------------------------------------------------------------
    # After printing all section rows, print another separator line for a neat ending.
    print("-" * (name_col_width + 50))
    
    # Finally, print the total runtime for the entire process.
    # The label 'Total Runtime' is left-aligned in the section column,
    # and the total runtime value is formatted to three decimal places.
    print(f"{'Total Runtime':<{name_col_width}}                       {total_time:.3f} seconds\n")
    # End of the print_time_summary function.


In [ ]:
# --------------------------------------------------------------
# Cell 3 : Horizons API Functions
# --------------------------------------------------------------
# This cell contains functions to interact with the JPL Horizons API.
# The Horizons API provides astronomical ephemeris data (e.g., positions, brightness)
# for solar system bodies. The functions below help in:
#   1. Constructing the POST data payload to send to Horizons.
#   2. Parsing the returned text data to extract specific values.
#   3. Managing the overall process of sending the request and retrieving results.

def create_input_content(dateobs, timeobs):
    """
    Build the text content needed to POST a request to JPL Horizons.
    
    This function constructs a multi-line string (which serves as a configuration file)
    containing various commands and parameters required by the Horizons API to generate
    ephemeris data. Specifically, it sets:
    
      - COMMAND='599': This selects the target object, here '599' corresponds to Jupiter.
      - OBJ_DATA='YES': Indicates that object-specific data should be returned.
      - MAKE_EPHEM='YES': Instructs Horizons to generate ephemeris data.
      - TABLE_TYPE='OBSERVER': Specifies that the output should be in an observer-centric format.
      - CENTER='500@399': Defines the observer’s location. The string '500@399' typically 
        denotes a geocentric viewpoint (or whichever center is chosen).
      - TLIST='{dateobs} {timeobs}': Sets the date and time for which the ephemeris is needed.
      - QUANTITIES='9,20,23,24': Specifies which quantities to include in the output:
          * 9  => Visual magnitude and surface brightness.
          * 20 => Observer range and range-rate.
          * 23 => Sun-Observer-Target (SOT) elongation.
          * 24 => Phase angle.
    
    Parameters:
    -----------
    dateobs : str
        The observation date, typically in the format 'YYYY-MM-DD' (e.g., '2021-05-03').
    timeobs : str
        The observation time, typically in the format 'HH:MM:SS' (e.g., '12:34:56').
    
    Returns:
    --------
    str
        A multi-line string containing the necessary commands and parameters, formatted
        in a way that the Horizons API can interpret when sent as the 'input.txt' file.
    """
    # The returned string is formatted as an f-string to dynamically include the provided date and time.
    # The special markers !$$SOF and !$$EOF denote the Start and End Of File, required by Horizons.
    return f"""
    !$$SOF
    COMMAND='599'
    OBJ_DATA='YES'
    MAKE_EPHEM='YES'
    TABLE_TYPE='OBSERVER'
    CENTER='500@399'
    TLIST='{dateobs} {timeobs}'
    QUANTITIES='9,20,23,24'
    !$$EOF
    """
    # This string, once generated, is ready to be attached as a file in a POST request to the API.


def parse_horizons_text_for_delta_and_sbrt(horizons_text):
    """
    Given the raw text response from Horizons, extract the observer distance (delta)
    and the surface brightness (S-brt) from the ephemeris data block.
    
    The Horizons response text contains a section delineated by $$SOE (Start Of Ephemeris)
    and $$EOE (End Of Ephemeris). This block contains the actual data rows with multiple columns.
    
    Expected token breakdown in a data row:
      - tokens[0] : Date/Time part 1
      - tokens[1] : Date/Time part 2
      - tokens[2] : Apparent magnitude (APmag)
      - tokens[3] : Surface brightness (S-brt)
      - tokens[4] : Observer distance (delta, in astronomical units, AU)
      - tokens[5] : Range rate (deldot)
      - tokens[6] : Sun-Observer-Target (SOT) elongation
      - tokens[7] : (Often a formatting token, e.g., "/r")
      - tokens[8] : Another token (e.g., S-T-O)
    
    The function will:
      1. Split the entire response text into individual lines.
      2. Locate the indices of the lines containing '$$SOE' and '$$EOE'.
      3. Extract the lines between these markers, which contain the ephemeris data.
      4. Iterate over each non-empty line in the data block:
           - Split the line into tokens.
           - If the token list has fewer than 9 elements, skip it.
           - Otherwise, attempt to convert token[3] and token[4] into floats:
             * token[3] -> s_brt (surface brightness)
             * token[4] -> delta (observer distance in AU)
      5. Return the parsed (delta, s_brt) values.
    
    If the expected markers are not found or if parsing fails, the function returns (None, None).
    
    Parameters:
    -----------
    horizons_text : str
        The complete raw text output from the Horizons API.
    
    Returns:
    --------
    tuple:
        A tuple (delta, s_brt) where:
         - delta : float or None, representing the observer distance in AU.
         - s_brt : float or None, representing the surface brightness.
    """
    # Split the entire Horizons text into a list of individual lines.
    lines = horizons_text.splitlines()
    
    try:
        # Identify the index of the line that contains '$$SOE', indicating the start of ephemeris data.
        start_index = next(i for i, line in enumerate(lines) if '$$SOE' in line)
        # Similarly, identify the index of the line that contains '$$EOE', indicating the end of ephemeris data.
        end_index   = next(i for i, line in enumerate(lines) if '$$EOE' in line)
    except StopIteration:
        # If either '$$SOE' or '$$EOE' is not found, the generator expression will raise StopIteration.
        # In that case, print an error message and return (None, None) to indicate failure.
        print("Could not find $$SOE/$$EOE in Horizons output.")
        return None, None

    # Extract the ephemeris data block by taking the lines that fall between the '$$SOE' and '$$EOE' markers.
    # Note: start_index+1 skips the '$$SOE' marker line; end_index is not included (i.e., it stops before '$$EOE').
    ephem_lines = lines[start_index+1 : end_index]

    # Loop through each line in the extracted data block.
    for ln in ephem_lines:
        # Remove any leading or trailing whitespace to ensure clean tokenization.
        ln = ln.strip()
        # If the line is empty (i.e., it contains no characters after stripping), skip it.
        if not ln:
            continue  # Continue to the next line if this one is blank.
        # Split the line into tokens based on whitespace. Each token should correspond to a data field.
        tokens = ln.split()

        # Validate that we have the minimum expected number of tokens (9 in this case).
        if len(tokens) < 9:
            continue  # If there are fewer than 9 tokens, skip this line as it doesn't contain all required data.
        try:
            # Parse the surface brightness from token[3]:
            s_brt  = float(tokens[3])
            # Parse the observer distance (delta) from token[4]:
            dist_au = float(tokens[4])
            # If both conversions are successful, return the parsed values.
            return dist_au, s_brt
        except ValueError:
            # If there is a ValueError during conversion (e.g., non-numeric value encountered),
            # skip this line and continue to the next one.
            continue

    # If no valid data line is found after processing all lines, return (None, None) to indicate failure.
    return None, None


def get_horizons_data(dateobs, timeobs):
    """
    Uses create_input_content() to build the Horizons POST data, sends the POST request,
    and returns the parsed observer distance (delta) and surface brightness (s_brt).
    
    Process Overview:
      1. Generate the POST payload using the provided date and time.
      2. Send a POST request to the Horizons API using the global 'horizons_url'.
         - The 'data' parameter specifies the response format as text.
         - The 'files' parameter attaches the generated content as 'input.txt'.
      3. Check the HTTP response status:
         - If the status code is not 200 (OK), print an error message and return (None, None).
      4. If successful, pass the response text to parse_horizons_text_for_delta_and_sbrt()
         to extract the desired parameters.
      5. Return the extracted (delta, s_brt) tuple.
    
    Parameters:
    -----------
    dateobs : str
        The observation date in 'YYYY-MM-DD' format (e.g., '2021-05-03').
    timeobs : str
        The observation time in 'HH:MM:SS' format (e.g., '12:34:56').
    
    Returns:
    --------
    tuple:
        (delta, s_brt) where:
         - delta : float or None, representing the observer's distance in AU.
         - s_brt : float or None, representing the surface brightness.
    """
    # Build the POST data payload by calling create_input_content with the specified date and time.
    # The resulting 'content' variable is a multi-line string formatted for the Horizons API.
    content = create_input_content(dateobs, timeobs)
    
    # Send the POST request to the Horizons API using the global URL (horizons_url).
    # The request is configured with:
    #   - A 'data' field specifying that the desired output format is plain text.
    #   - A 'files' field attaching the generated content as if it were a file named 'input.txt'.
    resp = requests.post(
        horizons_url,  # Global variable representing the API endpoint.
        data={'format': 'text'},  # Request parameter indicating output should be in text format.
        files={'input': ('input.txt', content)}  # Attach the generated content as a file.
    )

    # Check if the HTTP response status code is not 200 (which indicates an error in the request).
    if resp.status_code != 200:
        # Print an error message including the problematic status code.
        print(f"Failed Horizons request: {resp.status_code}")
        # Return (None, None) to signal that the data could not be retrieved.
        return None, None
    
    # If the request was successful (HTTP 200 OK), parse the response text to extract 'delta' and 's_brt'.
    # The function parse_horizons_text_for_delta_and_sbrt handles the extraction.
    return parse_horizons_text_for_delta_and_sbrt(resp.text)
    # End of get_horizons_data function.


In [ ]:
# --------------------------------------------------------------
# Cell 4 : Image Processing Utilities
# --------------------------------------------------------------

# Import OpenCV (cv2) to enable advanced image processing operations.
# OpenCV is widely used for tasks such as image transformation, filtering, and more.
import cv2  # Ensure OpenCV is available

def elliptical_mask(shape, center, major_axis, minor_axis, angle_deg):
    """
    Create a boolean mask (2D numpy array of True/False) indicating which pixels
    fall inside a given ellipse.

    Parameters:
    -----------
    shape : tuple of (height, width)
        The dimensions of the image or array on which the mask will be applied.
    center : tuple of (cx, cy)
        The (x, y) coordinates of the center of the ellipse.
    major_axis : float
        The full length (diameter) along the ellipse's major axis.
    minor_axis : float
        The full length (diameter) along the ellipse's minor axis.
    angle_deg : float
        The rotation angle of the ellipse in degrees.
        The angle is measured from the x-axis in the OpenCV coordinate convention.

    Returns:
    --------
    numpy.ndarray
        A 2D boolean numpy array of the same shape as specified, where:
          - True  indicates that the pixel lies inside the ellipse.
          - False indicates that the pixel lies outside the ellipse.
    """
    # Unpack the 'shape' tuple into individual dimensions:
    # 'h' represents the height (number of rows) and 'w' represents the width (number of columns).
    (h, w) = shape

    # Create coordinate grids using numpy's ogrid.
    # np.ogrid is used for generating open multi-dimensional "meshgrids", which is memory efficient.
    # y_grid: a column vector with values ranging from 0 to h-1 (each row index).
    # x_grid: a row vector with values ranging from 0 to w-1 (each column index).
    y_grid, x_grid = np.ogrid[0:h, 0:w]

    # Extract the center coordinates (cx, cy) from the provided 'center' tuple.
    # 'cx' is the x-coordinate and 'cy' is the y-coordinate of the ellipse's center.
    cx, cy = center

    # Convert the provided rotation angle from degrees to radians.
    # This is necessary because the trigonometric functions in numpy (np.cos, np.sin)
    # expect the angle in radians.
    theta = np.deg2rad(angle_deg)

    # Compute the semi-axis lengths:
    # The ellipse's major_axis and minor_axis are given as full diameters.
    # We need the semi-major axis (half of the major axis) and semi-minor axis (half of the minor axis).
    a = major_axis / 2.0  # Semi-major axis: half the length of the major axis.
    b = minor_axis / 2.0  # Semi-minor axis: half the length of the minor axis.

    # Shift the coordinate grids so that the origin (0,0) is moved to the ellipse's center (cx, cy).
    # This simplifies the subsequent computations, as the ellipse equation assumes the ellipse is centered at the origin.
    x_shifted = x_grid - cx  # Subtract the x-coordinate of the center from each x value.
    y_shifted = y_grid - cy  # Subtract the y-coordinate of the center from each y value.

    # Pre-calculate the cosine and sine of the rotation angle.
    # These values are used in the rotation transformation of the coordinate system.
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)

    # Rotate the shifted coordinate system by the specified angle.
    # This transformation aligns the ellipse with the coordinate axes, allowing us to apply the standard ellipse formula.
    # The rotation is performed using the following standard formulas:
    #   x_prime = x_shifted * cos(theta) + y_shifted * sin(theta)
    #   y_prime = -x_shifted * sin(theta) + y_shifted * cos(theta)
    x_prime =  x_shifted * cos_t + y_shifted * sin_t
    y_prime = -x_shifted * sin_t + y_shifted * cos_t

    # Apply the standard ellipse equation:
    #   (x_prime^2) / (a^2) + (y_prime^2) / (b^2) <= 1
    # This equation defines the set of points (x_prime, y_prime) that lie inside or on the boundary of the ellipse.
    ellipse_equation = (x_prime**2) / (a * a) + (y_prime**2) / (b * b)

    # Return a boolean array where each element is True if the corresponding pixel
    # satisfies the ellipse equation (i.e., lies inside the ellipse), and False otherwise.
    return ellipse_equation <= 1.0


def rotate_image_full(data, cx, cy, angle_deg):
    """
    Rotate the entire 2D array `data` (e.g., an image) around the specified center (cx, cy)
    by a given angle in degrees, using OpenCV's warpAffine function. The rotated image
    retains the same dimensions as the input.

    Parameters:
    -----------
    data : numpy.ndarray
        A 2D numpy array representing the image or data to rotate.
    cx, cy : float
        The x and y coordinates of the center of rotation.
    angle_deg : float
        The rotation angle in degrees. In OpenCV, positive angles indicate counter-clockwise rotation.

    Returns:
    --------
    numpy.ndarray
        A new 2D numpy array containing the rotated image/data, with the same dimensions as the original.
    """
    # Retrieve the dimensions of the input data array.
    # 'rows' is the number of rows (height) and 'cols' is the number of columns (width).
    rows, cols = data.shape

    # Create a 2x3 rotation matrix using OpenCV's cv2.getRotationMatrix2D function.
    # This matrix encapsulates both the rotation and translation required to perform the rotation.
    # Parameters for getRotationMatrix2D:
    #   - The center of rotation, provided as a tuple (cx, cy).
    #   - The rotation angle in degrees.
    #   - A scale factor (set to 1.0 for no scaling).
    M = cv2.getRotationMatrix2D((cx, cy), angle_deg, 1.0)

    # Apply the affine transformation to the input data using cv2.warpAffine.
    # This function transforms the image using the rotation matrix 'M'.
    # The third parameter (cols, rows) defines the size of the output image, which is kept the same.
    # The 'flags' parameter, set to cv2.INTER_LINEAR, uses linear interpolation to compute pixel values.
    rotated = cv2.warpAffine(data, M, (cols, rows), flags=cv2.INTER_LINEAR)

    # Return the resulting rotated image.
    return rotated


In [ ]:
# --------------------------------------------------------------
# Cell 5 : Main Analysis / Pipeline
# --------------------------------------------------------------

# Start the overall script timer to measure total execution time.
# "Script Start" is a label that helps identify this overall timing record.
start_section("Script Start")
# --------------------------------------------------------------------------------
# The above call to start_section() records the current time into the global
# time_records list with the label "Script Start" to later compute the total runtime.

# -----------------------------------------
# A) Get all FITS files
# -----------------------------------------
# Start a timer for the section that searches for FITS files.
start_section("Glob FITS files")

# Use the glob module to search recursively for all FITS files in the specified directory.
# The pattern indicates:
#   - "jupiter_data_WFC3_F631DRC/MAST_2023_08_29T0311/HST/" is the base directory.
#   - The first "*" matches any subdirectory inside HST.
#   - The second "*" matches any subdirectory inside that subdirectory.
#   - "*.fits" matches any file ending with the .fits extension.
fits_files = glob.glob("jupiter_data_WFC3_F631DRC/MAST_2023_08_29T0311/HST/*/*.fits")
# --------------------------------------------------------------------------------
# At this point, fits_files is a list containing the file paths to all .fits files
# matching the above pattern.

# End the timer for "Glob FITS files" once the search is complete.
end_section()

# Check if any FITS files were found.
if len(fits_files) == 0:
    # If no files are found, notify the user.
    print("No FITS files found in the specified directory.")
    # End the overall timer (or this section) and print the timing summary.
    end_section()
    print_time_summary()
    # Optionally, you might want to exit the script if no files are found.
    # raise SystemExit("No FITS files found.")  # Optionally exit
else:
    # If FITS files are found, print the total count.
    print(f"Found {len(fits_files)} FITS files.\n")
# --------------------------------------------------------------------------------
# This branch ensures that the rest of the processing only occurs if there are files.

# -----------------------------------------
# B) Read FITS metadata
# -----------------------------------------
# Create an empty list to store metadata and image data from each FITS file.
file_details = []
# Start the timer for reading FITS metadata.
start_section("Reading FITS metadata")

# Loop over each FITS file found in the previous step.
for fits_file in fits_files:
    try:
        # Open the FITS file using astropy.io.fits in a context manager.
        # The 'with' statement ensures the file is properly closed after processing.
        with fits.open(fits_file) as hdul:
            # 'hdul' is an HDUList (list of Header/Data Units).
            # Typically, hdul[0] is the primary HDU (contains header and sometimes data),
            # and hdul[1] is expected to contain the actual image data.
            header = hdul[0].header   # Extract the primary header.
            raw_data = hdul[1].data   # Extract the image data from the first extension.

            # Check if the raw_data is valid. If not, create a minimal placeholder array.
            if raw_data is None:
                # Create a 1x1 array of type float32 filled with 0.0.
                data = np.zeros((1, 1), dtype=np.float32)
            else:
                # Replace any NaN or infinite values with 0.0 to avoid issues in computations.
                data = np.nan_to_num(raw_data, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)

            # Compute the total flux by summing all pixel values.
            total_flux = float(np.sum(data))

            # Extract key metadata from the FITS header.
            # If a key is missing, use 'N/A' as a default.
            file_name = os.path.basename(fits_file)  # Get just the filename.
            date_obs = header.get('DATE-OBS', 'N/A')   # Observation date.
            time_obs = header.get('TIME-OBS', 'N/A')     # Observation time.
            telescope = header.get('TELESCOP', 'N/A')    # Telescope used.
            instrument = header.get('INSTRUME', 'N/A')   # Instrument used.
            exposure_time = header.get('EXPTIME', 'N/A')   # Exposure duration.
            t_filter = header.get('FILTER', 'N/A')         # Filter information.

            # Append a tuple with all the relevant information to file_details.
            file_details.append((
                file_name,      # Name of the file.
                data,           # Processed image data.
                total_flux,     # Computed total flux.
                date_obs,       # Date of observation.
                time_obs,       # Time of observation.
                telescope,      # Telescope name.
                instrument,     # Instrument name.
                exposure_time,  # Exposure time.
                t_filter        # Filter used.
            ))
    except Exception as e:
        # If an error occurs while processing a file, print the error and continue with the next file.
        print(f"Error processing file {fits_file}: {e}")

# End the timer for "Reading FITS metadata" after processing all files.
end_section()

# -----------------------------------------
# C) Sort the file_details by DATE-OBS & TIME-OBS
# -----------------------------------------
# Start the timer for sorting the file details.
start_section("Sort file_details")

# Sort the file_details list based on observation date and time.
# The lambda function extracts the date (element index 3) and time (element index 4)
# to be used as the sorting keys.
file_details = sorted(file_details, key=lambda x: (x[3], x[4]))

# End the timer for the sorting operation.
end_section()

# -----------------------------------------
# D) Process images: rotate, find ellipse flux
# -----------------------------------------
# Start a timer for the entire image processing section.
start_section("Processing images")

# Initialize an empty list to store computed metrics (summaries) for each image.
# Each summary will be a tuple containing various values such as file number,
# surface brightness, measured flux values, and observation details.
ellipse_summaries = []
# The format of each tuple is:
# (file_num, f_name, s_brt, e_flux, delta, ef_err, f_avg, fa_err, d_obs, t_obs)

# Iterate over each file detail along with its index (starting at 1).
for i, (file_name, data, total_flux, date_obs, time_obs,
        telescope, instrument, exposure_time, t_filter) in enumerate(file_details, start=1):

    # Start a timer for processing the individual file.
    start_section(f"Process {file_name}")

    #========================#
    # 1) Get Horizons Data
    #========================#
    # Initialize variables to hold Horizons data:
    # dist_au: Distance from observer in astronomical units.
    # s_brt: Surface brightness value from the Horizons API.
    dist_au = None
    s_brt = None
    # Only query Horizons if both observation date and time are valid.
    if date_obs != 'N/A' and time_obs != 'N/A':
        # Call the get_horizons_data() function to fetch data from JPL Horizons.
        # It returns a tuple: (distance in AU, surface brightness value).
        dist_au, s_brt_val = get_horizons_data(date_obs, time_obs)
        # If a valid surface brightness value is returned, assign it.
        if s_brt_val:
            s_brt = s_brt_val

    #========================#
    # 2) Build Original 8-bit Image
    #========================#
    # Convert the original image data to an 8-bit format suitable for display.
    if data.size > 1:
        # Compute the 99th percentile of the pixel values to determine a high cutoff.
        high_cut_orig = np.percentile(data, 99)
        # Clip the data to the range [0, high_cut_orig] to suppress extreme outliers.
        clipped_orig = np.clip(data, 0, high_cut_orig)
        # Normalize the clipped data to a 0-255 range and convert to 8-bit unsigned integers.
        norm_8u_orig = cv2.normalize(clipped_orig, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    else:
        # If the data array is invalid or has only one pixel, create a default black image.
        norm_8u_orig = np.zeros((1, 1), dtype=np.uint8)

    #========================#
    # 3) Rotate the image
    #========================#
    # Set a default rotation angle.
    rotate_angle = 0.0
    # Compute the center of the image (cx, cy) to use as the pivot for rotation.
    cx, cy = (data.shape[1] / 2.0, data.shape[0] / 2.0)

    # Apply a low-level threshold to the original 8-bit image to isolate significant features.
    tmp_thresh_val = 5  # A low threshold value to separate background from the object.
    # cv2.threshold returns a tuple, where the second element is the binary image.
    _, tmp_thresh = cv2.threshold(norm_8u_orig, tmp_thresh_val, 255, cv2.THRESH_BINARY)

    # Find external contours in the binary image.
    tmp_contours, _ = cv2.findContours(tmp_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # If any contours are found, use the largest one to estimate the rotation.
    if len(tmp_contours) > 0:
        # Select the contour with the maximum area.
        largest_tmp_contour = max(tmp_contours, key=cv2.contourArea)
        # Fit an ellipse to the largest contour if it has at least 5 points (a requirement for ellipse fitting).
        if len(largest_tmp_contour) >= 5:
            # cv2.fitEllipse returns the center, size (width, height), and rotation angle of the ellipse.
            (temp_cx, temp_cy), (w, h), angle = cv2.fitEllipse(largest_tmp_contour)
            # If the ellipse appears vertical (height greater than width), adjust the angle by adding 90°.
            if h > w:
                angle += 90
                # Swap width and height to reflect the adjustment.
                w, h = h, w
            # Set the rotation angle for the image to the fitted ellipse angle.
            rotate_angle = angle

    # Rotate the original image data using our rotate_image_full() function.
    # This rotates the image around the center (cx, cy) by the computed rotate_angle.
    rotated_data = rotate_image_full(data, cx, cy, rotate_angle)
    # Compute the total flux in the rotated image by summing its pixel values.
    rotated_flux = float(np.sum(rotated_data))

    #=====================================================#
    # 4) Detect ellipse on rotated data, measure flux
    #=====================================================#
    # Process the rotated image to detect the elliptical region of interest.
    # Compute the 99th percentile to define a high cutoff for the rotated image.
    high_cut_rot = np.percentile(rotated_data, 99)
    # Clip the rotated data to suppress extreme values.
    clipped_rot = np.clip(rotated_data, 0, high_cut_rot)
    # Normalize the clipped data to an 8-bit scale.
    norm_8u_rot = cv2.normalize(clipped_rot, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Apply a binary threshold to the normalized rotated image.
    threshold_value = 5
    _, thr_rot = cv2.threshold(norm_8u_rot, threshold_value, 255, cv2.THRESH_BINARY)
    # Find external contours in the thresholded rotated image.
    rot_contours, _ = cv2.findContours(thr_rot, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize variables for flux measurements and error estimates.
    ellipse_flux = 0.0  # Flux within the fitted ellipse.
    cropped_flux = 0.0  # Flux within the bounding rectangle around the contour.
    flux_avg = 0.0      # A computed flux average (flux per unit exposure and distance).
    ef_err = 0.0        # Error estimate for ellipse flux (using Poisson statistics).
    fa_err = 0.0        # Error estimate for flux average.

    # Create an overlay image for visualization by converting the grayscale rotated image
    # into a color (BGR) image. This allows colored drawing of contours and ellipses.
    overlay_img = cv2.cvtColor(norm_8u_rot, cv2.COLOR_GRAY2BGR)

    # If contours were detected in the rotated image, proceed to analyze them.
    if len(rot_contours) > 0:
        # Identify the largest contour by area.
        rot_largest_contour = max(rot_contours, key=cv2.contourArea)
        # If the largest contour has enough points, fit an ellipse to it.
        if len(rot_largest_contour) >= 5:
            # Fit an ellipse and retrieve its center, size, and rotation angle.
            (rcx, rcy), (rw, rh), rangle = cv2.fitEllipse(rot_largest_contour)
            # Adjust the ellipse parameters if it is oriented vertically.
            if rh > rw:
                rangle += 90
                rw, rh = rh, rw
            # Generate a boolean mask for pixels inside the fitted ellipse.
            mask_ell = elliptical_mask(rotated_data.shape, (rcx, rcy), rw, rh, rangle)
            # Sum the flux (pixel values) inside the elliptical region.
            ellipse_flux = float(np.sum(rotated_data[mask_ell]))
            # Draw the fitted ellipse on the overlay image in green (color code: (0, 255, 0)).
            cv2.ellipse(overlay_img, ((rcx, rcy), (rw, rh), rangle), (0, 255, 0), 2)

        # Also compute a bounding rectangle around the largest contour.
        rx, ry, rW, rH = cv2.boundingRect(rot_largest_contour)
        # Crop the rotated image to the area defined by the bounding rectangle.
        cropped_data = rotated_data[ry:ry + rH, rx:rx + rW]
        # Compute the total flux in the cropped region.
        cropped_flux = float(np.sum(cropped_data))
        # Draw the bounding rectangle on the overlay image in blue (color code: (255, 0, 0)).
        cv2.rectangle(overlay_img, (rx, ry), (rx + rW, ry + rH), (255, 0, 0), 2)

        # Estimate the error in the ellipse flux measurement using Poisson statistics.
        # The error is approximated as the square root of the ellipse flux.
        ef_err = np.sqrt(ellipse_flux) if ellipse_flux > 0 else 0.0

        # Compute the flux average if valid exposure time and distance are available.
        # The flux average is defined as ellipse_flux divided by (exposure_time * dist_au).
        if exposure_time not in [None, 'N/A', 0] and dist_au:
            flux_avg = ellipse_flux / (float(exposure_time) * dist_au)
            # Propagate the error estimate similarly.
            fa_err = ef_err / (float(exposure_time) * dist_au)

    #========================#
    # 5) Save row in summaries
    #========================#
    # Append a tuple with all computed parameters for the current image to the ellipse_summaries list.
    ellipse_summaries.append((
        i,                  # file_num: Sequential number of the file.
        file_name,          # f_name: Name of the FITS file.
        s_brt,              # s_brt: Surface brightness from Horizons data.
        ellipse_flux,       # e_flux: Flux measured within the ellipse.
        dist_au,            # delta: Distance from observer (in AU) from Horizons data.
        ef_err,             # ef_err: Error in ellipse flux.
        flux_avg,           # f_avg: Computed flux average.
        fa_err,             # fa_err: Error in flux average.
        date_obs,           # d_obs: Observation date.
        time_obs            # t_obs: Observation time.
    ))

    #--------------------------------#
    # 6) Print parameter-value table
    #--------------------------------#
    # Create a list of tuples where each tuple contains a parameter label and its value.
    param_values = [
        ("File #",           str(i)),
        ("File Name",        file_name),
        ("DATE-OBS",         date_obs),
        ("TIME-OBS",         time_obs),
        ("Telescope",        telescope),
        ("Instrument",       instrument),
        ("Filter",           t_filter),
        ("surface brightness (Jupiter)",
         f"{s_brt:.3f}" if s_brt else "N/A"),
        ("Total Flux",       f"{total_flux:.2f}"),
        ("Flux (Rotated)",   f"{rotated_flux:.2f}"),
        ("Ellipse Flux",     f"{ellipse_flux:.2f}"),
        ("Cropped Flux",     f"{cropped_flux:.2f}"),
        ("Exposure Time",    f"{exposure_time}"),
        ("Delta (AU)",       f"{dist_au:.5f}" if dist_au else "N/A"),
        ("Flux Average",     f"{flux_avg:.2f}")
    ]

    # Print a header and separator for clarity.
    print("\n" + "=" * 60)
    print(f"  Parameter-Value Table for {file_name}")
    print("=" * 60)
    # Define a column width for proper alignment of parameter labels.
    col_width = 35
    # Loop over each parameter and its corresponding value and print them.
    for (p, v) in param_values:
        print(f"{p:<{col_width}} {v}")
    print("=" * 60)

    #--------------------------------#
    # 7) Visualization: 4-panel plot
    #--------------------------------#
    # Create a figure with 4 subplots arranged in a 1x4 grid.
    # figsize specifies the overall dimensions of the figure.
    fig, axes = plt.subplots(1, 4, figsize=(22, 5))

    # Panel (A): Display the original normalized 8-bit image.
    axes[0].imshow(norm_8u_orig, cmap='gray', origin='lower')
    axes[0].set_title(f"Normalized 8-bit\n(Total Flux={total_flux:.2f})", fontsize=10)
    axes[0].axis('off')  # Remove axis ticks and labels.

    # Panel (B): Display the rotated image.
    axes[1].imshow(norm_8u_rot, cmap='gray', origin='lower')
    axes[1].set_title(f"Rotated Angle={rotate_angle:.1f}°\n(Flux={rotated_flux:.2f})", fontsize=10)
    axes[1].axis('off')

    # Panel (C): Display the overlay image showing the detected contour and fitted ellipse.
    axes[2].imshow(overlay_img, origin='lower')
    axes[2].set_title(f"Contour+Ellipse\n(EllipseFlux={ellipse_flux:.2f})", fontsize=10)
    axes[2].axis('off')

    # Panel (D): Display the cropped region.
    if cropped_data.size > 1:
        # For the cropped data, compute the 99th percentile cutoff.
        high_cut_crop = np.percentile(cropped_data, 99)
        # Clip and normalize the cropped image to 8-bit.
        clipped_crop = np.clip(cropped_data, 0, high_cut_crop)
        norm_8u_crop = cv2.normalize(clipped_crop, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        axes[3].imshow(norm_8u_crop, cmap='gray', origin='lower')
        axes[3].set_title(f"Cropped\n(Flux={cropped_flux:.2f})", fontsize=10)
        axes[3].axis('off')
    else:
        # If no cropped data is found, fallback to displaying the rotated image.
        axes[3].imshow(norm_8u_rot, cmap='gray', origin='lower')
        axes[3].set_title("Cropped\n(No contour found)", fontsize=10)
        axes[3].axis('off')

    # Set a super title for the figure that displays the current file name.
    plt.suptitle(f"File: {file_name}", fontsize=12)
    # Adjust the layout to prevent overlap between subplots.
    plt.tight_layout()
    # Display the figure.
    plt.show()

    # End the timer for processing this specific file.
    end_section()  # end "Process {file_name}"

# End the timer for the "Processing images" section.
end_section()  # end "Processing images"

# End the overall script timer.
end_section()  # end "Script Start"

# After all processing is complete, print a structured summary of timing for all sections.
print_time_summary()
# --------------------------------------------------------------------------------
# This final call outputs a table summarizing the start time, end time, and duration
# for each timed section, as well as the total runtime of the entire script.


In [ ]:
#-------------------------------
# TEST CASE Cell 6 : Debug / Test Horizons
#-------------------------------
def test_horizons_api(file_index=0):
    """
    Debug function to demonstrate the Horizons API response for one selected FITS file from `file_details`.

    Steps:
      1) Show relevant FITS header info (DATE-OBS, TIME-OBS).
      2) Display the exact input content being sent to Horizons.
      3) Retrieve the full text response from Horizons and print it.
      4) Demonstrate how the parser extracts delta and S-brt from that text.
      5) Compare with the standard get_horizons_data() function output.

    Parameters:
    -----------
    file_index : int
        The index into our file_details list. Default=0 means the first file.
    """
    # ------------------------------------------------------------------------------
    # Step 0: Basic Safety Check
    # Before proceeding, ensure that the global list 'file_details' exists and is populated.
    # If it's empty, that means the main analysis hasn't been run, so we abort.
    if not file_details:
        print("No file_details found. Make sure you've run the main analysis cell(s).")
        return

    # Check if the provided file_index is within the valid range.
    if file_index < 0 or file_index >= len(file_details):
        print(f"Invalid file_index {file_index}; valid range is [0..{len(file_details)-1}]")
        return

    # ------------------------------------------------------------------------------
    # Step 1: Extract Relevant Information from the Selected FITS File
    # Unpack the tuple from file_details corresponding to the specified file_index.
    # The tuple contains: (file_name, data, total_flux, date_obs, time_obs, telescope, instrument, exposure_time, t_filter)
    (
        file_name,      # The name of the FITS file (extracted from the file path).
        data,           # The image data array from the FITS file.
        total_flux,     # The total flux computed from the image data.
        date_obs,       # The observation date (from the FITS header).
        time_obs,       # The observation time (from the FITS header).
        telescope,      # Telescope information (from the FITS header).
        instrument,     # Instrument information (from the FITS header).
        exposure_time,  # Exposure time (from the FITS header).
        t_filter        # Filter used (from the FITS header).
    ) = file_details[file_index]

    # Print out the extracted FITS file information in a formatted manner.
    print("=== Selected FITS File Information ===")
    print(f"Index:         {file_index}")
    print(f"File Name:     {file_name}")
    print(f"DATE-OBS:      {date_obs}")
    print(f"TIME-OBS:      {time_obs}")
    print(f"Telescope:     {telescope}")
    print(f"Instrument:    {instrument}")
    print(f"Filter:        {t_filter}")
    print(f"Exposure Time: {exposure_time}")
    print("======================================\n")

    # ------------------------------------------------------------------------------
    # Step 2: Display the Exact Content Being Sent to Horizons
    # Call the create_input_content() function with the observation date and time.
    # This function returns a multi-line string that contains all the commands and parameters
    # formatted for the Horizons API request.
    horizons_input = create_input_content(date_obs, time_obs)
    
    # Print out the generated input content so that the user can inspect exactly what is sent.
    print("=== Content Sent to Horizons API ===")
    print(horizons_input)
    print("=====================================\n")

    # ------------------------------------------------------------------------------
    # Step 3: Retrieve and Print the Full Text Response from Horizons
    # Import the requests module here (if not already imported globally) for making HTTP requests.
    import requests

    # Define the URL endpoint for the Horizons API.
    horizons_url_debug = 'https://ssd.jpl.nasa.gov/api/horizons_file.api'

    # Send a POST request to the Horizons API.
    # Parameters:
    #   - data={'format': 'text'} requests the response in plain text format.
    #   - files={'input': ('input.txt', horizons_input)} attaches the input content as a file.
    debug_response = requests.post(
        horizons_url_debug,
        data={'format': 'text'},
        files={'input': ('input.txt', horizons_input)}
    )

    # Check if the request was successful by verifying the HTTP status code.
    if debug_response.status_code != 200:
        # If the request failed (status code is not 200), print an error message.
        print(f"Failed Horizons request: {debug_response.status_code}")
        return

    # If successful, retrieve the full text response from the API.
    horizons_full_text = debug_response.text

    # Print the full text response for debugging purposes.
    print("=== Full Horizons API Response ===")
    print(horizons_full_text)
    print("==================================\n")

    # ------------------------------------------------------------------------------
    # Step 4: Demonstrate How to Parse Out 'delta' and 'S-brt' from the Response
    # Split the full text response into individual lines.
    lines = horizons_full_text.splitlines()
    
    try:
        # Locate the line indices where the ephemeris data begins and ends.
        # '$$SOE' marks the Start Of Ephemeris and '$$EOE' marks the End Of Ephemeris.
        start_index = next(i for i, line in enumerate(lines) if '$$SOE' in line)
        end_index   = next(i for i, line in enumerate(lines) if '$$EOE' in line)
    except StopIteration:
        # If either marker is not found, parsing cannot proceed.
        print("Could not find $$SOE/$$EOE in Horizons output—parsing aborted.")
        return

    # Extract only the lines that contain the ephemeris data.
    ephem_lines = lines[start_index+1 : end_index]

    # Initialize variables to hold the extracted observer distance and surface brightness.
    dist_au_extracted = None  # Will store the extracted delta (distance in AU).
    s_brt_extracted   = None  # Will store the extracted surface brightness (S-brt).
    relevant_line     = None  # Will store the line used for parsing (for display purposes).

    # Iterate through each line in the ephemeris block.
    for ln in ephem_lines:
        ln = ln.strip()  # Remove leading/trailing whitespace.
        if not ln:
            continue  # Skip empty lines.
        # Split the line into tokens (individual data columns).
        tokens = ln.split()
        # Save the current line as the one we are using for parsing.
        relevant_line = ln
        # The expected format is that tokens[3] holds S-brt and tokens[4] holds delta.
        try:
            s_brt_extracted = float(tokens[3])  # Convert S-brt to a float.
            dist_au_extracted = float(tokens[4])  # Convert delta to a float.
        except:
            # If conversion fails, simply pass; in a robust implementation, you might log this.
            pass
        # Break after processing the first non-empty line for demonstration purposes.
        break

    # Print out the parsing demonstration results.
    print("=== Parsing Ephemeris Lines ===")
    if relevant_line:
        print(f"Line used for parsing:\n{relevant_line}\n")
        print(f"Extracted distance (delta) [OLD WAY]: {dist_au_extracted}")
        print(f"Extracted surface brightness (S-brt) [OLD WAY]: {s_brt_extracted}")
    else:
        print("No valid line found with ephemeris data.")
    print("================================\n")

    # ------------------------------------------------------------------------------
    # Step 5: Compare with the Official get_horizons_data() Function Output
    # Call the standard get_horizons_data() function to retrieve parsed values.
    debug_delta, debug_sbrt = get_horizons_data(date_obs, time_obs)
    # Print the results from get_horizons_data() for comparison.
    print("=== Comparison with get_horizons_data() ===")
    print(f"[CORRECT] get_horizons_data returned delta = {debug_delta}, s_brt = {debug_sbrt}")
    print("===========================================\n")

# ------------------------------------------------------------------------------
# Example usage: Call the test function for the first file (file_index=0)
test_horizons_api(file_index=0)


In [ ]:
# --------------------------------------------------------------
# Cell 7 : Final Data Summaries & Plots
# --------------------------------------------------------------
# This cell is responsible for building a summary table from the processed
# image data stored in the global variable `ellipse_summaries` and printing it
# with nicely formatted, auto-adjusted, centered columns.
# It also includes a helper function `print_dynamic_table` to handle the printing.

def print_dynamic_table(rows):
    """
    Print a table with auto-adjusted column widths and centered text.

    Parameters:
    -----------
    rows : list of lists
        A list where each sub-list represents one row of the table.
        Typically, the first row (rows[0]) is the header row containing column titles.

    Process:
    --------
    1. Convert every item in every row to a string (ensuring uniform text output).
    2. Determine the maximum string length for each column so that the printed
       table has columns with appropriate widths.
    3. Print the header row with text centered in each column.
    4. Print a separator line below the header.
    5. Print the remaining rows (data rows) with each cell centered.
    """
    # --------------------------------------------------------------------------
    # Step 1: Convert all items in the table to strings.
    # This ensures that regardless of the original data type (int, float, etc.),
    # each item will be printed as text.
    str_rows = [[str(item) for item in row] for row in rows]
    # The above is a nested list comprehension that iterates over each row in 'rows',
    # and then over each item in the row, converting it to a string.

    # --------------------------------------------------------------------------
    # Step 2: Determine the maximum width needed for each column.
    # We assume all rows have the same number of columns, so we take the length of the first row.
    num_cols = len(str_rows[0])
    # Create a list 'col_widths' where each element corresponds to the maximum string length
    # found in that column across all rows.
    col_widths = [max(len(row[i]) for row in str_rows) for i in range(num_cols)]
    # For each column index i (ranging from 0 to num_cols-1), we:
    #   - Iterate over each row in 'str_rows'
    #   - Calculate the length of the string at position i in that row
    #   - Take the maximum length found; this is the width for column i.

    # --------------------------------------------------------------------------
    # Step 3: Print the header row (which is assumed to be the first row in the table).
    header_row = str_rows[0]
    # Build a separator line (a list of strings) for each column by repeating '-' for the column's width.
    sep_line = ["-" * col_widths[i] for i in range(num_cols)]
    # Print the header row with each cell centered within its column.
    # The join() function is used to combine the centered cells with two spaces in between.
    print("  ".join(cell.center(col_widths[i]) for i, cell in enumerate(header_row)))
    # Print the separator line below the header row.
    print("  ".join(sep_line))

    # --------------------------------------------------------------------------
    # Step 4: Print each data row (all rows except the first one) with centered text.
    for row in str_rows[1:]:
        # For each row, center each cell within the predetermined column width,
        # then join them together with two spaces between cells.
        print("  ".join(cell.center(col_widths[i]) for i, cell in enumerate(row)))


# ------------------------------
# 1. BUILD THE TABLE
# ------------------------------
# We now build the data for our table, starting with the header row.
# This header row defines the titles for each column.
table_rows = [
    [
        "File #",         # Column 1: File number (sequential index).
        "File Name",      # Column 2: Name of the FITS file.
        "SB (Jupiter)",   # Column 3: Surface brightness for Jupiter (from Horizons API).
        "Ellipse Flux",   # Column 4: Flux measured within the detected ellipse.
        "delta",          # Column 5: Observer distance (delta) from Horizons API.
        "Error Bars EF",  # Column 6: Error estimate for the ellipse flux.
        "Flux Average",   # Column 7: Computed flux average.
        "Error Bars FA",  # Column 8: Error estimate for the flux average.
        "DATE-OBS",       # Column 9: Observation date.
        "TIME-OBS"        # Column 10: Observation time.
    ]
]

# ------------------------------------------------------------------------------
# Now we loop over the global 'ellipse_summaries' list to fill in the data rows.
# Each entry in ellipse_summaries is a tuple with the following structure:
# (file_num, f_name, s_brt, e_flux, delta, ef_err, f_avg, fa_err, d_obs, t_obs)
for (file_num, f_name, s_brt, e_flux, delta, ef_err, f_avg, fa_err, d_obs, t_obs) in ellipse_summaries:
    
    # ------------------------------------------------------------------------------
    # Convert each numeric value to a formatted string.
    # If the value is None or evaluates to False, substitute with "N/A".
    # This ensures that missing or invalid data is clearly indicated.
    s_brt_str   = f"{s_brt:.3f}"   if s_brt   else "N/A"  # Format surface brightness to 3 decimal places.
    e_flux_str  = f"{e_flux:.2f}"   if e_flux  else "N/A"  # Format ellipse flux to 2 decimal places.
    delta_str   = f"{delta:.5f}"    if delta   else "N/A"  # Format delta (observer distance) to 5 decimal places.
    ef_err_str  = f"{ef_err:.2f}"   if ef_err  else "N/A"  # Format error for ellipse flux to 2 decimal places.
    f_avg_str   = f"{f_avg:.3f}"    if f_avg   else "N/A"  # Format flux average to 3 decimal places.
    fa_err_str  = f"{fa_err:.3f}"   if fa_err  else "N/A"  # Format error for flux average to 3 decimal places.

    # ------------------------------------------------------------------------------
    # Build a new row (a list of strings) corresponding to the current entry.
    # The new row has the same order of fields as defined in the header.
    new_row = [
        str(file_num),   # Convert file number to string.
        f_name,          # File name (already a string).
        s_brt_str,       # Formatted surface brightness.
        e_flux_str,      # Formatted ellipse flux.
        delta_str,       # Formatted observer distance.
        ef_err_str,      # Formatted error in ellipse flux.
        f_avg_str,       # Formatted flux average.
        fa_err_str,      # Formatted error in flux average.
        d_obs,           # Observation date.
        t_obs            # Observation time.
    ]
    # Append the newly created row to the table_rows list.
    table_rows.append(new_row)

# ------------------------------------------------------------------------------
# Finally, print the entire table using the helper function.
# This function will auto-adjust the column widths and center the text.
print_dynamic_table(table_rows)


In [ ]:
# ------------------------------
# Cell 8 : The sun graph (Modified for dot markers for apparent magnitude)
# ------------------------------

# Import the requests module to handle HTTP requests.
import requests

# ------------------------------------------------------------------------------
# Define the base URL for the Horizons API.
# This URL is used to request ephemeris data in a file-based text format.
url = "https://ssd.jpl.nasa.gov/api/horizons_file.api"

# ------------------------------------------------------------------------------
# Build the input content that will be sent to the Horizons API.
# This multi-line string contains specific commands and parameters that instruct
# Horizons on what data to return.
#
# In this example:
#   - COMMAND='10' selects the Sun (object number 10 in Horizons).
#   - CENTER='500@-48' specifies the observer's location (here, a particular center).
#   - MAKE_EPHEM='YES' tells Horizons to generate an ephemeris.
#   - EPHEM_TYPE='OBSERVER' requests observer-centric ephemeris data.
#   - START_TIME and STOP_TIME define the date range for the ephemeris.
#   - STEP_SIZE='60m' sets the time step between ephemeris entries (every 60 minutes).
#   - QUANTITIES='9,20' selects specific output quantities:
#         * Quantity 9: Typically the apparent magnitude.
#         * Quantity 20: Typically the observer distance (delta) and range rate.
#   - OUT_UNITS='KM-S' sets the output units (here kilometers and seconds).
#   - CSV_FORMAT='NO' specifies that the output should not be in CSV format.
#   - The special markers !$$SOF and !$$EOF denote the Start and End Of File.
input_content = """
!$$SOF
COMMAND='10'
CENTER='500@-48'
MAKE_EPHEM='YES'
EPHEM_TYPE='OBSERVER'
START_TIME='2015-01-19 00:00:00'
STOP_TIME='2019-07-21 23:59:00'
STEP_SIZE='60m'
QUANTITIES='9,20'
OUT_UNITS='KM-S'
CSV_FORMAT='NO'
!$$EOF
"""

# ------------------------------------------------------------------------------
# Define additional parameters for the POST request.
# Here we indicate that we want the response in plain text format.
params = {"format": "text"}

# ------------------------------------------------------------------------------
# Send the POST request to the Horizons API.
# The request is composed of:
#   - 'url': The API endpoint.
#   - 'data': A dictionary with additional parameters (format=text).
#   - 'files': The input content is sent as if it were a file named 'input.txt'.
response = requests.post(url, data=params, files={'input': ('input.txt', input_content)})

# ------------------------------------------------------------------------------
# Split the response text into individual lines.
# This makes it easier to search for the ephemeris data block.
lines = response.text.splitlines()

# ------------------------------------------------------------------------------
# Locate the ephemeris data block within the response text.
# The block of data is delimited by lines containing "$$SOE" (Start Of Ephemeris)
# and "$$EOE" (End Of Ephemeris).
start_idx = None  # Will hold the index of the line containing "$$SOE"
end_idx = None    # Will hold the index of the line containing "$$EOE"
for i, line in enumerate(lines):
    if "$$SOE" in line:
        start_idx = i  # Found the start marker
    if "$$EOE" in line:
        end_idx = i    # Found the end marker
        break        # Once the end marker is found, exit the loop

# ------------------------------------------------------------------------------
# Prepare empty lists to store the parsed data:
#   - timestamps: The date and time of each ephemeris entry.
#   - apmag_vals: The apparent magnitude values of the Sun.
#   - delta_vals: The observer distance (delta) values.
timestamps = []
apmag_vals = []
delta_vals = []

# ------------------------------------------------------------------------------
# Check that both start and end markers were found before proceeding.
if start_idx is not None and end_idx is not None:
    # Iterate over the lines within the ephemeris data block.
    # We skip the marker line itself by starting at start_idx+1, and we stop at end_idx.
    for line in lines[start_idx+1:end_idx]:
        line = line.strip()  # Remove any leading/trailing whitespace.
        if not line:
            continue  # Skip empty lines to avoid processing blank data.
        tokens = line.split()  # Split the line into individual data tokens based on whitespace.
        # Expected token positions based on QUANTITIES='9,20' are:
        #   tokens[0]: Date (string)
        #   tokens[1]: Time (string)
        #   tokens[2]: Apparent magnitude (APmag)
        #   tokens[4]: Observer distance (delta)
        if len(tokens) >= 5:
            # Concatenate the date and time tokens into one datetime string.
            dt_str = tokens[0] + " " + tokens[1]
            try:
                # Convert the datetime string into a pandas datetime object.
                dt = pd.to_datetime(dt_str)
                # Convert the apparent magnitude and delta tokens into floats.
                apmag = float(tokens[2])  # Apparent magnitude of the Sun.
                delta = float(tokens[4])  # Observer distance.
                # Append the parsed values to their corresponding lists.
                timestamps.append(dt)
                apmag_vals.append(apmag)
                delta_vals.append(delta)
            except Exception as e:
                # If parsing fails for any reason (e.g., conversion error), print an error message.
                print(f"Skipping line due to parse error: {line}")
                continue  # Move on to the next line.

# ------------------------------------------------------------------------------
# Once parsing is complete, create a pandas DataFrame to conveniently work with the data.
# The DataFrame will contain columns for datetime, apparent magnitude (apmag), and delta.
df = pd.DataFrame({
    "datetime": timestamps,
    "apmag": apmag_vals,
    "delta": delta_vals
})

# Sort the DataFrame by the datetime column to ensure the data is in chronological order.
df.sort_values("datetime", inplace=True)

# ------------------------------------------------------------------------------
# Calculate the number of days since the first ephemeris entry.
# This is useful for plotting the data on an x-axis that represents elapsed time.
start_time = df["datetime"].min()  # The earliest datetime in the DataFrame.
# Compute a new column "days_since_start" as the difference in seconds divided by 86400 (seconds per day).
df["days_since_start"] = (df["datetime"] - start_time).dt.total_seconds() / 86400.0

# ------------------------------------------------------------------------------
# Create a dual-axis plot to display both delta and apparent magnitude versus time.
# The left y-axis will show delta (observer distance), and the right y-axis will show
# apparent magnitude (apmag) with dot markers.
fig, ax_left = plt.subplots(figsize=(12, 6))
# Create a secondary y-axis sharing the same x-axis.
ax_right = ax_left.twinx()

# ------------------------------------------------------------------------------
# Plot delta as a blue line on the left y-axis.
ax_left.plot(df["days_since_start"], df["delta"], color='blue', label='Delta')
ax_left.set_xlabel("Days Since Start")             # Label for the x-axis.
ax_left.set_ylabel("Delta", color='blue')            # Label for the left y-axis.
ax_left.tick_params(axis='y', labelcolor='blue')     # Set the tick labels color for clarity.

# ------------------------------------------------------------------------------
# Plot apparent magnitude as red scatter points on the right y-axis.
# The marker size 's' is set to 0.5 to create tiny dots.
ax_right.scatter(df["days_since_start"], df["apmag"], color='red', s=0.5, label='Apparent Mag')
ax_right.set_ylabel("Apparent Magnitude", color='red')  # Label for the right y-axis.
ax_right.tick_params(axis='y', labelcolor='red')         # Set tick labels to red.

# ------------------------------------------------------------------------------
# Add a title to the overall plot that explains the displayed data.
plt.title("Apparent Magnitude (Right Axis) and Delta (Left Axis) vs. Time")

# Display the plot.
plt.show()


In [ ]:
# ------------------------------
# Cell 9 : 2. TIME vs. DELTA, Average Flux, and SB vs. Time Plots (Side by Side)
# ------------------------------

# Import the pandas library for data manipulation and matplotlib.pyplot for plotting.
import pandas as pd
import matplotlib.pyplot as plt

# --------------------------------------------------------------------------------
# This cell builds time-series data for three different metrics extracted from the
# global variable `ellipse_summaries`:
#   1) Distance (delta) vs. time
#   2) Average Flux vs. time
#   3) Surface Brightness vs. time
#
# Each of these is first stored in a list of records, then converted into a pandas
# DataFrame for sorting, manipulation, and plotting.
# --------------------------------------------------------------------------------

# ================================
# 1) Build Distance (delta) vs. Time Data
# ================================

# Initialize an empty list to store records for delta (observer distance) data.
# Each record will be a tuple: (datetime, delta)
delta_records = []

# Loop over each entry in the global ellipse_summaries list.
# Each entry is a tuple with the following structure:
# (file_num, f_name, s_brt, e_flux, dist_au, ef_err, f_avg, fa_err, d_obs, t_obs)
for (file_num, f_name, s_brt, e_flux, dist_au, ef_err, f_avg, fa_err, d_obs, t_obs) in ellipse_summaries:
    # Only process records that have valid observation date and time.
    if d_obs != "N/A" and t_obs != "N/A":
        # Combine the date and time strings into a single datetime string.
        dt_str = d_obs + " " + t_obs
        # Parse the datetime string into a pandas datetime object.
        dt_parsed = pd.to_datetime(dt_str)
        # Append a tuple (datetime, delta) to the list.
        delta_records.append((dt_parsed, dist_au))

# Create a DataFrame from the list of delta records.
# The DataFrame will have two columns: "datetime" and "delta_au".
df_delta = pd.DataFrame(delta_records, columns=["datetime", "delta_au"])
# Sort the DataFrame by the datetime column to ensure the records are in chronological order.
df_delta.sort_values("datetime", inplace=True)

# Convert the datetime column to "days since first observation":
#   - Find the earliest datetime value.
#   - Subtract that value from each datetime to compute elapsed time in seconds.
#   - Convert seconds to days by dividing by 86400 (number of seconds in a day).
start_time = df_delta["datetime"].min()
df_delta["days_since_start"] = (df_delta["datetime"] - start_time).dt.total_seconds() / 86400.0

# ================================
# 2) Build Average Flux vs. Time Data
# ================================

# Initialize an empty list to store records for average flux data.
# Each record will be a tuple: (datetime, f_avg)
flux_records = []

# Loop over each entry in ellipse_summaries, similar to the previous block.
for (file_num, f_name, s_brt, e_flux, dist_au, ef_err, f_avg, fa_err, d_obs, t_obs) in ellipse_summaries:
    # Only process records with valid observation date and time.
    if d_obs != "N/A" and t_obs != "N/A":
        # Combine the date and time into one string.
        dt_str = d_obs + " " + t_obs
        # Parse the combined string into a datetime object.
        dt_parsed = pd.to_datetime(dt_str)
        # Append a tuple (datetime, average flux) to the list.
        flux_records.append((dt_parsed, f_avg))

# Create a DataFrame from the flux records with columns "datetime" and "f_avg".
df_flux = pd.DataFrame(flux_records, columns=["datetime", "f_avg"])
# Sort the DataFrame chronologically.
df_flux.sort_values("datetime", inplace=True)
# Compute days since the first observation for the flux data.
start_time = df_flux["datetime"].min()  # Note: This start_time may be different from df_delta if records differ.
df_flux["days_since_start"] = (df_flux["datetime"] - start_time).dt.total_seconds() / 86400.0

# ================================
# 3) Build Surface Brightness vs. Time Data
# ================================

# Initialize an empty list to store records for surface brightness data.
# Each record will be a tuple: (datetime, s_brt)
sb_records = []

# Loop over each entry in ellipse_summaries.
for (file_num, f_name, s_brt, e_flux, dist_au, ef_err, f_avg, fa_err, d_obs, t_obs) in ellipse_summaries:
    # Process only if observation date and time are valid.
    if d_obs != "N/A" and t_obs != "N/A":
        # Create a datetime string by concatenating the date and time.
        dt_str = d_obs + " " + t_obs
        # Convert the datetime string into a pandas datetime object.
        dt_parsed = pd.to_datetime(dt_str)
        # Append a tuple (datetime, surface brightness) to the list.
        sb_records.append((dt_parsed, s_brt))

# Create a DataFrame from the surface brightness records with columns "datetime" and "s_brt".
df_sb = pd.DataFrame(sb_records, columns=["datetime", "s_brt"])
# Sort the DataFrame by datetime.
df_sb.sort_values("datetime", inplace=True)
# Compute the days since the first observation for the surface brightness data.
start_time = df_sb["datetime"].min()
df_sb["days_since_start"] = (df_sb["datetime"] - start_time).dt.total_seconds() / 86400.0

# --------------------------------------------------------------------------------
# Create a 3-panel side-by-side figure to display the three metrics as separate subplots.
# The figure will have 3 columns (one per metric) and a specified figure size.
fig, axs = plt.subplots(ncols=3, figsize=(18, 6))

# ------------------------------------------
# Plot 1: Delta (Distance) vs. Time
# ------------------------------------------
# Create a scatter plot using the "days_since_start" on the x-axis and "delta_au" on the y-axis.
axs[0].scatter(df_delta["days_since_start"], df_delta["delta_au"], color='black', alpha=0.75)
axs[0].set_xlabel("Days Since Start")       # Set the x-axis label.
axs[0].set_ylabel("Delta (AU)")               # Set the y-axis label.
axs[0].set_title("Distance (Delta) vs. Time (Days)")  # Set the subplot title.

# ------------------------------------------
# Plot 2: Average Flux vs. Time
# ------------------------------------------
# Create a scatter plot for average flux.
axs[1].scatter(df_flux["days_since_start"], df_flux["f_avg"], color='blue', alpha=0.75)
axs[1].set_xlabel("Days Since Start")         # Label for the x-axis.
axs[1].set_ylabel("Average Flux")             # Label for the y-axis.
axs[1].set_title("Average Flux vs. Time (Days)")  # Set the subplot title.

# ------------------------------------------
# Plot 3: Surface Brightness vs. Time
# ------------------------------------------
# Create a scatter plot for surface brightness.
axs[2].scatter(df_sb["days_since_start"], df_sb["s_brt"], color='green', alpha=0.75)
axs[2].set_xlabel("Days Since Start")         # Label for the x-axis.
axs[2].set_ylabel("Surface Brightness")       # Label for the y-axis.
axs[2].set_title("Surface Brightness vs. Time (Days)")  # Set the subplot title.

# Adjust layout spacing to prevent overlapping of subplots.
plt.tight_layout()
# Display the 3-panel figure.
plt.show()

# --------------------------------------------------------------------------------
# Additional Overlay Plots with Dual y-Axes:
# These plots overlay two metrics on the same x-axis (Days Since Start)
# but use two different y-axes (one on the left and one on the right).
# --------------------------------------------------------------------------------

# -----------------------------------------------------------
# Overlay i): Delta & Average Flux vs. Time
# -----------------------------------------------------------
# Create a new figure for the overlay plot.
fig, ax1 = plt.subplots(figsize=(12, 6))
# Plot delta as a scatter plot on the primary y-axis.
ax1.scatter(df_delta["days_since_start"], df_delta["delta_au"], color='black', alpha=0.75, label='Delta')
ax1.set_xlabel("Days Since Start")         # Set x-axis label.
ax1.set_ylabel("Delta (AU)", color='black')  # Set primary y-axis label.
ax1.tick_params(axis='y', labelcolor='black')  # Color the y-axis ticks for clarity.

# Create a secondary y-axis that shares the same x-axis.
ax1_right = ax1.twinx()
# Plot average flux on the secondary y-axis.
ax1_right.scatter(df_flux["days_since_start"], df_flux["f_avg"], color='blue', alpha=0.75, label='Average Flux')
ax1_right.set_ylabel("Average Flux", color='blue')  # Set secondary y-axis label.
ax1_right.tick_params(axis='y', labelcolor='blue')   # Color the ticks accordingly.

# Set a title for the overlay plot.
plt.title("Overlay: Delta and Average Flux vs. Time")
plt.tight_layout()
plt.show()

# -----------------------------------------------------------
# Overlay ii): Delta & Surface Brightness vs. Time
# -----------------------------------------------------------
# Create a new figure for the next overlay.
fig, ax2 = plt.subplots(figsize=(12, 6))
# Plot delta (distance) on the primary y-axis.
ax2.scatter(df_delta["days_since_start"], df_delta["delta_au"], color='black', alpha=0.75, label='Delta')
ax2.set_xlabel("Days Since Start")
ax2.set_ylabel("Delta (AU)", color='black')
ax2.tick_params(axis='y', labelcolor='black')

# Create a secondary y-axis for surface brightness.
ax2_right = ax2.twinx()
# Plot surface brightness on the secondary y-axis.
ax2_right.scatter(df_sb["days_since_start"], df_sb["s_brt"], color='green', alpha=0.75, label='Surface Brightness')
ax2_right.set_ylabel("Surface Brightness", color='green')
ax2_right.tick_params(axis='y', labelcolor='green')

# Set the title and adjust layout.
plt.title("Overlay: Delta and Surface Brightness vs. Time")
plt.tight_layout()
plt.show()

# -----------------------------------------------------------
# Overlay iii): Average Flux & Surface Brightness vs. Time
# -----------------------------------------------------------
# Create a new figure for the third overlay plot.
fig, ax3 = plt.subplots(figsize=(12, 6))
# Plot average flux on the primary y-axis.
ax3.scatter(df_flux["days_since_start"], df_flux["f_avg"], color='blue', alpha=0.75, label='Average Flux')
ax3.set_xlabel("Days Since Start")
ax3.set_ylabel("Average Flux", color='blue')
ax3.tick_params(axis='y', labelcolor='blue')

# Create a secondary y-axis for surface brightness.
ax3_right = ax3.twinx()
# Plot surface brightness on the secondary y-axis.
ax3_right.scatter(df_sb["days_since_start"], df_sb["s_brt"], color='green', alpha=0.75, label='Surface Brightness')
ax3_right.set_ylabel("Surface Brightness", color='green')
ax3_right.tick_params(axis='y', labelcolor='green')

# Set title and adjust the layout.
plt.title("Overlay: Average Flux and Surface Brightness vs. Time")
plt.tight_layout()
plt.show()


In [ ]:
# ------------------------------
# Cell 10 : quadruple overlay
# ------------------------------

# Import required modules for advanced multi-axis plotting:
#   - host_subplot: Allows creating a host axes with attached parasite axes.
#   - mpl_toolkits.axisartist (imported as AA): Provides support for customizing axis artists.
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

# Import matplotlib.pyplot for plotting functions and numpy for numerical operations.
import matplotlib.pyplot as plt
import numpy as np

# --------------------------------------------------------------------------------
# The goal of this cell is to create a combined multi-axis plot that overlays
# several time-series data sets on a single time axis (x-axis) while assigning
# multiple y-axes (each with its own scale and label). The overlays include:
#
#   - Sun Delta (line plot, taken from DataFrame "df")
#   - Sun Apparent Magnitude (scatter plot, from DataFrame "df")
#   - Jupiter Delta (scatter plot, from DataFrame "df_delta")
#   - Jupiter Average Flux (scatter plot, from DataFrame "df_flux")
#   - Jupiter Surface Brightness (scatter plot, from DataFrame "df_sb")
#
# Note: The DataFrames (df, df_delta, df_flux, df_sb) are assumed to be defined earlier.
# --------------------------------------------------------------------------------

# Create a new figure with a specified size.
fig = plt.figure(figsize=(14, 8))

# Create the "host" axes using host_subplot.
# This axes will act as the base (or host) for the multiple overlaid y-axes.
# The parameter 111 indicates a single subplot.
# The axes_class=AA.Axes argument allows us to use the AxisArtist toolkit for custom axis management.
host = host_subplot(111, axes_class=AA.Axes)

# Adjust the right boundary of the host subplot to leave room for additional y-axes.
plt.subplots_adjust(right=0.75)

# --------------------------------------------------------------------------------
# Create "parasite" y-axes.
# Each additional axis is created as a twin of the host. 
# They will later be offset from the default position to avoid overlapping.
# --------------------------------------------------------------------------------

# par1 will be used for the Sun Apparent Magnitude.
par1 = host.twinx()

# par2 will be used for Jupiter Delta.
par2 = host.twinx()

# par3 will be used for Jupiter Average Flux.
par3 = host.twinx()

# par4 will be used for Jupiter Surface Brightness.
par4 = host.twinx()

# --------------------------------------------------------------------------------
# Offset the extra parasite axes to the right.
# By default, all twinx() axes appear on the right. We use the new_fixed_axis method
# to create additional axes at specified offsets.
# The offset is specified as a tuple (offset, 0) in points.
# --------------------------------------------------------------------------------

# Start with an offset of 60 points.
offset = 60

# For par2:
#   - Get the grid helper from par2 and create a new fixed axis on the right.
#   - Set its offset to (offset, 0) to move it to the right by 60 points.
new_fixed_axis = par2.get_grid_helper().new_fixed_axis
par2.axis["right"] = new_fixed_axis(loc="right", axes=par2, offset=(offset, 0))

# Increment the offset by 60 for the next parasite axis.
offset += 60

# For par3:
#   - Create a new fixed axis and set its offset accordingly.
new_fixed_axis = par3.get_grid_helper().new_fixed_axis
par3.axis["right"] = new_fixed_axis(loc="right", axes=par3, offset=(offset, 0))

# Increment the offset again.
offset += 60

# For par4:
#   - Create a new fixed axis with the updated offset.
new_fixed_axis = par4.get_grid_helper().new_fixed_axis
par4.axis["right"] = new_fixed_axis(loc="right", axes=par4, offset=(offset, 0))

# --------------------------------------------------------------------------------
# Now plot each of the data series on their respective axes.
# It is assumed that the following DataFrames exist:
#   - df: Contains Sun data with columns "days_since_start", "delta", and "apmag"
#   - df_delta: Contains Jupiter Delta data with columns "days_since_start", "delta_au"
#   - df_flux: Contains Jupiter Average Flux data with columns "days_since_start", "f_avg"
#   - df_sb: Contains Jupiter Surface Brightness data with columns "days_since_start", "s_brt"
# --------------------------------------------------------------------------------

# Plot Sun Delta on the host axes as a blue line.
# p1 will hold the line object for potential legend creation.
p1, = host.plot(df["days_since_start"], df["delta"], color='blue', label='Sun Delta')

# Plot Sun Apparent Magnitude on the first parasite axis (par1) as red scatter points.
# The marker size is set to 1 for small dots.
p2 = par1.scatter(df["days_since_start"], df["apmag"], color='red', s=1, label='Sun Apparent Mag')

# Plot Jupiter Delta on the second parasite axis (par2) as black scatter points.
p3 = par2.scatter(df_delta["days_since_start"], df_delta["delta_au"], color='black', alpha=0.75, label='Jupiter Delta')

# Plot Jupiter Average Flux on the third parasite axis (par3) as blue scatter points.
p4 = par3.scatter(df_flux["days_since_start"], df_flux["f_avg"], color='blue', alpha=0.75, label='Jupiter Avg Flux')

# Plot Jupiter Surface Brightness on the fourth parasite axis (par4) as green scatter points.
p5 = par4.scatter(df_sb["days_since_start"], df_sb["s_brt"], color='green', alpha=0.75, label='Jupiter SB')

# --------------------------------------------------------------------------------
# Set labels for the x-axis and each y-axis:
#   - The host x-axis is labeled "Days Since Start".
#   - The host y-axis (left) represents Sun Delta.
#   - Each parasite axis gets its own y-axis label.
# --------------------------------------------------------------------------------

# Set x-axis label on the host.
host.set_xlabel("Days Since Start")
# Set host y-axis label (for Sun Delta).
host.set_ylabel("Sun Delta")
# Set label for par1 (Sun Apparent Magnitude).
par1.set_ylabel("Sun Apparent Mag")
# Set label for par2 (Jupiter Delta).
par2.set_ylabel("Jupiter Delta")
# Set label for par3 (Jupiter Avg Flux).
par3.set_ylabel("Jupiter Avg Flux")
# Set label for par4 (Jupiter Surface Brightness).
par4.set_ylabel("Jupiter Surface Brightness")

# --------------------------------------------------------------------------------
# Color code the axis labels for clarity:
#   - The host left axis label is colored blue.
#   - The parasite axes on the right are colored with their corresponding series colors.
# --------------------------------------------------------------------------------

host.axis["left"].label.set_color('blue')      # Sun Delta (blue)
par1.axis["right"].label.set_color('red')         # Sun Apparent Magnitude (red)
par2.axis["right"].label.set_color('black')       # Jupiter Delta (black)
par3.axis["right"].label.set_color('blue')        # Jupiter Avg Flux (blue)
par4.axis["right"].label.set_color('green')       # Jupiter Surface Brightness (green)

# --------------------------------------------------------------------------------
# Customize x-axis tick locations:
# Calculate the maximum number of days across all data sets to set the x-axis ticks.
max_days = max(df["days_since_start"].max(),
               df_delta["days_since_start"].max(),
               df_flux["days_since_start"].max(),
               df_sb["days_since_start"].max())
# Set x-axis ticks at intervals of 100 days from 0 to max_days + 100.
host.set_xticks(np.arange(0, max_days + 100, 100))

# --------------------------------------------------------------------------------
# Optionally, you can build a combined legend.
# The following commented-out code shows how to collect line objects and labels,
# then pass them to host.legend() for a unified legend.
#
# lines = [p1, p2, p3, p4, p5]
# labels = [l.get_label() for l in lines]
# host.legend(lines, labels, loc='upper left')
# --------------------------------------------------------------------------------

# Set a title for the overall plot.
plt.title("Combined Quadruple Axis Plot (5 Y-Axes) vs. Time")

# Draw and display the final plot.
plt.draw()
plt.show()


In [ ]:
# -------------------------------------------------------------------------
# Cell 11 (Enhanced) : Diagnostic & Debugging System for DBSCAN Clusters 
#                      (Average Flux) WITH "index_number" COLUMN ADDED
# -------------------------------------------------------------------------
#
# Description:
#   This cell builds upon the previous diagnostics code for analyzing 
#   clustering in df_flux. The key difference here is that we now create a 
#   new column called "index_number", which enumerates each row in the 
#   cluster data. In both the snippet (.head(5)) and the full cluster dump, 
#   you'll see that column so you can track each row easily.
#
#   Otherwise, the layout, logic, and step-by-step debugging prints remain 
#   the same. We haven't removed any existing information — only added 
#   the "index_number" column to the cluster data output.
#
# Usage:
#   - Ensure df_flux is defined with columns ['datetime','f_avg','days_since_start'].
#   - Run this cell. The new run_dbscan_diagnostics function 
#     will provide thorough debugging output plus an "index_number" column.
# -------------------------------------------------------------------------

# Import necessary libraries for numerical operations, data handling, plotting,
# and clustering.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN  # Import the DBSCAN clustering algorithm from scikit-learn

def run_dbscan_diagnostics(
    df_flux,
    eps=0.25,
    min_samples=3,
    margin_scale=0.1
):
    """
    Performs a comprehensive diagnostic analysis on df_flux for clustering 
    in the (days_since_start, f_avg) space.

    Parameters
    ----------
    df_flux : pd.DataFrame
        Must contain at least the following columns:
           - 'datetime': Timestamps of observations.
           - 'f_avg':    The average flux data.
           - 'days_since_start': The time offset (in days) from a reference time.
    eps : float
        The maximum distance between two samples for one to be considered 
        as in the neighborhood of the other (a DBSCAN parameter).
    min_samples : int
        The minimum number of samples required in a neighborhood for a point to be considered 
        a core point (a DBSCAN parameter).
    margin_scale : float
        Fractional padding added to each cluster's min/max time when setting subplot x-limits.
        For example, 0.1 adds 10% of the cluster's time span as padding.

    Returns
    -------
    None
        This function prints diagnostic information and displays plots.
    """
    # -------------------------------------------------------------------------
    # Start the diagnostic process by printing a header.
    print("\n===========================")
    print(" DBSCAN DIAGNOSTIC PROCESS")
    print("===========================\n")

    # -------------------------------------------------------------------------
    # STEP 1: Basic Checks on df_flux Structure
    print("STEP 1: Checking df_flux structure and required columns...")
    # Define the required columns that must be present in df_flux.
    required_cols = ["datetime", "f_avg", "days_since_start"]
    # Iterate through each required column and check if it exists in df_flux.
    for c in required_cols:
        if c not in df_flux.columns:
            # If a column is missing, print an error and abort the function.
            print(f"ERROR: Missing column '{c}' in df_flux! Cannot proceed.")
            print("Suggestion: Ensure df_flux has these columns or rename appropriately.")
            return  # Abort execution if required columns are not present

    # Print a sample of the first 5 rows of df_flux for verification.
    print("\ndf_flux sample (first 5 rows):")
    print(df_flux.head(5))

    # -------------------------------------------------------------------------
    # STEP 2: Validate for NaN or Infinite Values in Critical Columns
    print("\nSTEP 2: Validating for NaN or infinite values in [f_avg, days_since_start]...")
    # Count the number of NaN values in the 'f_avg' column.
    nan_count_favg = df_flux["f_avg"].isna().sum()
    # Count the number of NaN values in the 'days_since_start' column.
    nan_count_days = df_flux["days_since_start"].isna().sum()
    # If there are any NaN values, print a warning.
    if nan_count_favg > 0 or nan_count_days > 0:
        print(f"WARNING: Found {nan_count_favg} NaN in 'f_avg' and {nan_count_days} in 'days_since_start'.")
        print("         This could impact DBSCAN. Suggestion: remove or fill NaNs.")
    else:
        print("No NaN values found in the relevant columns.")

    # -------------------------------------------------------------------------
    # STEP 3: Check the Range of 'days_since_start'
    print("\nSTEP 3: Checking range of days_since_start...")
    # Get the minimum and maximum values of 'days_since_start'.
    days_min = df_flux["days_since_start"].min()
    days_max = df_flux["days_since_start"].max()
    print(f"days_since_start => min: {days_min:.3f}, max: {days_max:.3f}")
    # If the minimum is significantly negative (and by more than a year), warn the user.
    if days_min < 0 and abs(days_min) > 365:
        print("NOTE: There's a significantly negative min time. This could indicate multiple reference points.")
        print("      If unintended, double-check your reference_time assignment.")
    # If the maximum value is very large, warn the user.
    if days_max > 3000:
        print("NOTE: A very large days_since_start (>3000). Are you sure your reference is correct?")

    # -------------------------------------------------------------------------
    # STEP 4: Run DBSCAN Clustering on the 'days_since_start' Data
    print(f"\nSTEP 4: Running DBSCAN with eps={eps} and min_samples={min_samples}...")
    # Reshape the 'days_since_start' data into a 2D array, as required by scikit-learn.
    X = df_flux["days_since_start"].values.reshape(-1, 1)
    try:
        # Create a DBSCAN object with the given eps and min_samples parameters.
        db = DBSCAN(eps=eps, min_samples=min_samples)
        # Fit DBSCAN to the data and predict cluster labels for each point.
        cluster_labels = db.fit_predict(X)
    except Exception as e:
        # If an error occurs, print the error and abort.
        print(f"ERROR: DBSCAN failed with an exception: {e}")
        print("Suggestion: verify that your data is numeric and the DBSCAN parameters are valid.")
        return

    # Add a new column "cluster" to df_flux with the assigned cluster labels.
    df_flux["cluster"] = cluster_labels

    # Identify the unique clusters (excluding noise points labeled as -1).
    clusters = df_flux[df_flux["cluster"] != -1]["cluster"].unique()
    clusters.sort()

    print("DBSCAN completed successfully.")
    print(f"Number of data points: {len(df_flux)}")
    # Count the number of noise points (points with label -1).
    n_noise = sum(cluster_labels == -1)
    print(f"Number of noise points: {n_noise}")
    print(f"Cluster labels found (excluding -1): {clusters}")

    # -------------------------------------------------------------------------
    # STEP 5: Summarize Each Cluster
    print("\nSTEP 5: Summarizing clusters:")
    if len(clusters) == 0:
        print("No valid clusters were detected (all points might be noise).")
        print("Possible reasons or suggestions:")
        print(" - Increase eps to allow more points to form a cluster.")
        print(" - Decrease min_samples to allow smaller clusters.")
        print(" - Verify that days_since_start is correct.")
        return

    # Loop over each identified cluster label.
    for label in clusters:
        # Extract the subset of df_flux corresponding to the current cluster.
        cdata = df_flux[df_flux["cluster"] == label]
        # Determine the minimum and maximum 'days_since_start' values for the cluster.
        cmin = cdata["days_since_start"].min()
        cmax = cdata["days_since_start"].max()
        # Determine the size (number of points) of the cluster.
        csize = len(cdata)
        print(f"  Cluster {label}: size={csize}, time-range=({cmin:.2f}, {cmax:.2f})")

    # -------------------------------------------------------------------------
    # STEP 6: Create Subplots for Each Cluster with Debugging Information
    print("\nSTEP 6: Creating subplots for each cluster.")
    n_clusters = len(clusters)
    # Create a figure with a subplot for each cluster.
    # The height is set to 3 inches per cluster.
    fig, axs = plt.subplots(
        n_clusters, 
        1, 
        figsize=(12, 3 * n_clusters), 
        sharex=False
    )
    # If there is only one cluster, wrap the axis in a list for consistency.
    if n_clusters == 1:
        axs = [axs]

    # Loop over each cluster for plotting.
    for i, cluster_label in enumerate(clusters):
        ax = axs[i]
        # Select the data corresponding to the current cluster.
        cluster_data = df_flux[df_flux["cluster"] == cluster_label]

        # Create a copy of the cluster data after resetting its index.
        # This allows us to add a new column "index_number" without altering the original df_flux.
        temp_for_print = cluster_data.reset_index(drop=True).copy()
        # Add the "index_number" column. Here, cluster_data.index gives the original indices,
        # so adding 1 ensures the numbers start from 1.
        temp_for_print["index_number"] = cluster_data.index + 1

        # Print a header for the current cluster.
        print(f"\n  -> Plotting cluster {cluster_label + 1}: {len(cluster_data)} points.")

        # Print a snippet (first 5 rows) of the cluster data, showing only selected columns.
        # Use to_string(index=False) to skip printing the default DataFrame index.
        snippet_data = temp_for_print.head(5)
        print(snippet_data[["index_number","days_since_start","f_avg"]].to_string(index=False))

        # Print the full cluster data with the "index_number" column for complete diagnostics.
        print(f"\n     Full cluster data for cluster {cluster_label}:")
        print(temp_for_print[["index_number","days_since_start","f_avg"]].to_string(index=False))
        print("     -------------------------")

        # Plot the cluster data: 'days_since_start' on the x-axis and 'f_avg' on the y-axis.
        ax.scatter(
            cluster_data["days_since_start"], 
            cluster_data["f_avg"],
            color='blue', 
            alpha=0.75, 
            s=20
        )

        # Determine the x-axis limits for the plot by calculating the range of 'days_since_start'.
        t_min = cluster_data["days_since_start"].min()
        t_max = cluster_data["days_since_start"].max()
        span  = t_max - t_min
        if span == 0:
            # If all points have the same time, set a default span.
            span = 0.2
            t_min -= 0.1
            t_max += 0.1
        else:
            # Otherwise, add a margin proportional to the span.
            margin = margin_scale * span
            t_min -= margin
            t_max += margin

        # Apply the computed x-axis limits.
        ax.set_xlim(t_min, t_max)

        # Determine the y-axis limits for the average flux data.
        favg_min = cluster_data["f_avg"].min()
        favg_max = cluster_data["f_avg"].max()
        y_span = favg_max - favg_min
        if y_span == 0:
            # Set a fallback span if all flux values are identical.
            y_span = 1.0
            favg_min -= 0.5
            favg_max += 0.5
        else:
            # Add a margin to the y-axis limits.
            y_margin = margin_scale * y_span
            favg_min -= y_margin
            favg_max += y_margin

        # Apply the computed y-axis limits.
        ax.set_ylim(favg_min, favg_max)
        # Label the y-axis.
        ax.set_ylabel("Average Flux")
        # Set the title for the subplot, indicating the cluster number and time range.
        ax.set_title(f"Cluster {cluster_label + 1}: {t_min:.2f} to {t_max:.2f} days", fontsize=16)

    # Set the x-axis label for the last subplot.
    axs[-1].set_xlabel("Days Since Start")
    # Adjust subplot layout to avoid overlap.
    plt.tight_layout()
    # Display the subplots.
    plt.show()

    # Print a final message indicating the end of diagnostics.
    print("\nAll clusters plotted successfully. End of Cell 11 Diagnostics.\n")


# -------------------------------------------------------------------------
# Example usage:
# -------------------------------------------------------------------------
# Run the DBSCAN diagnostics function on df_flux with customized parameters.
# Here, eps=0.5, min_samples=1, and margin_scale=0.1 are passed to control clustering
# sensitivity and plot margins.
run_dbscan_diagnostics(df_flux, eps=0.5, min_samples=1, margin_scale=0.1)


In [ ]:
#sum of all the lines of code that have been written

import nbformat

# Replace with the path to your notebook file
notebook_path = 'Main Commented.ipynb'

# Read the notebook file using nbformat (version 4 is standard for recent notebooks)
nb = nbformat.read(notebook_path, as_version=4)

# Initialize a counter for code lines
total_code_lines = 0

# Loop over all cells in the notebook
for cell in nb.cells:
    # Process only code cells
    if cell.cell_type == 'code':
        # Split the cell source by newline to count the lines of code in that cell
        lines = cell.source.splitlines()
        total_code_lines += len(lines)

print(f"Total lines of code in the notebook: {total_code_lines}")
